# Teste do Gauss-Newton (IIR) — pydaptivefiltering

Este notebook gera um sinal de entrada `x[k]`, simula um sistema IIR estável conhecido (coeficientes verdadeiros),
e usa `GaussNewton.optimize()` para estimar os coeficientes.

Ele valida:
- contrato de I/O do `optimize` (`outputs`, `errors`, `coefficients`, metadados)
- `internal_states` quando `return_internal_states=True`
- comportamento esperado: erro reduz em média (início vs fim)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Ajuste o import conforme a estrutura real do seu pacote.
import pydaptivefiltering as pdf

In [3]:
def simulate_iir(x: np.ndarray, a: np.ndarray, b: np.ndarray) -> np.ndarray:
    """Simula um IIR no formato compatível com o GaussNewton.

    Modelo:
      y[k] = sum_{i=1..p} a[i-1] * y[k-i] + sum_{j=0..q} b[j] * x[k-j]
    """
    x = np.asarray(x, dtype=float)
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)

    p = a.size
    q = b.size - 1
    N = x.size

    y = np.zeros(N, dtype=float)
    for k in range(N):
        ar = 0.0
        for i in range(1, p + 1):
            if k - i >= 0:
                ar += a[i - 1] * y[k - i]

        ma = 0.0
        for j in range(0, q + 1):
            if k - j >= 0:
                ma += b[j] * x[k - j]

        y[k] = ar + ma
    return y

def mse(x: np.ndarray) -> float:
    x = np.asarray(x, dtype=float)
    return float(np.mean(x**2))

In [5]:
# Configuração do experimento
rng = np.random.default_rng(0)

N = 2500
zeros_order = 2   # q
poles_order = 2   # p

# Coeficientes verdadeiros (escolha estável: polos dentro do círculo unitário)
a_true = np.array([0.55, -0.15], dtype=float)         # tamanho p
b_true = np.array([0.80, -0.30, 0.12], dtype=float)   # tamanho (q+1)

x = rng.standard_normal(N).astype(float)
y_clean = simulate_iir(x, a_true, b_true)

noise_std = 0.02
d = y_clean + noise_std * rng.standard_normal(N)

print("MSE(y_clean):", mse(y_clean))
print("MSE(noise):  ", noise_std**2)

MSE(y_clean): 0.6609415915795972
MSE(noise):   0.0004


In [7]:
# Rodando o Gauss-Newton
gn = pdf.GaussNewton(
    zeros_order=zeros_order,
    poles_order=poles_order,
    alpha=0.05,
    step_size=1.0,
    delta=1e-3,
    w_init=None
)

res = gn.optimize(
    input_signal=x,
    desired_signal=d,
    verbose=True,
    return_internal_states=True
)

res.keys()

TypeError: GaussNewton.optimize() missing 1 required positional argument: 'x'

In [ ]:
# Checagens rápidas de contrato (I/O)
required_keys = ["outputs", "errors", "coefficients", "algo", "runtime_s", "error_kind"]
missing = [k for k in required_keys if k not in res]
assert not missing, f"Faltando chaves obrigatórias no retorno: {missing}"

y_hat = res["outputs"]
e = res["errors"]
W_hist = res["coefficients"]

assert y_hat.shape == (N,), f"outputs com shape inesperado: {y_hat.shape}"
assert e.shape == (N,), f"errors com shape inesperado: {e.shape}"

print("algo:", res["algo"])
print("runtime_s:", res["runtime_s"])
print("error_kind:", res["error_kind"])

# internal_states (opcional)
assert "internal_states" in res and res["internal_states"] is not None,     "Esperava internal_states quando return_internal_states=True"

st = res["internal_states"]
assert "x_sensitivity" in st and "y_sensitivity" in st, "internal_states incompleto"
assert st["x_sensitivity"].shape == (N,)
assert st["y_sensitivity"].shape == (N,)
print("internal_states OK:", list(st.keys()))

In [ ]:
# Métrica simples: o erro deve diminuir em média (comparando início vs fim)
k0 = int(0.1 * N)
k1 = int(0.9 * N)

mse_head = mse(e[:k0])
mse_tail = mse(e[k1:])

print("MSE erro (primeiros 10%):", mse_head)
print("MSE erro (últimos 10%):  ", mse_tail)

assert mse_tail < mse_head, "O erro não diminuiu (em média) do início para o fim."

In [ ]:
# Visualizações: saída e erro
plt.figure()
plt.plot(d, label="d (desejado)", linewidth=1)
plt.plot(y_hat, label="y_hat (estimado)", linewidth=1)
plt.title("Saída estimada vs desejada")
plt.legend()
plt.show()

plt.figure()
plt.plot(e, linewidth=1)
plt.title("Erro e[k] = d[k] - y_hat[k]")
plt.show()

In [ ]:
# Coeficientes: últimos valores vs verdade
# Nota: dependendo de como _record_history armazena, coefficients pode ser list/ndarray.
if isinstance(W_hist, list):
    W_arr = np.stack(W_hist, axis=0)
else:
    W_arr = np.asarray(W_hist)

print("W_arr shape:", W_arr.shape)

w_final = W_arr[-1]
w_true = np.concatenate([a_true, b_true])

print("w_final:", np.round(w_final, 5))
print("w_true: ", np.round(w_true, 5))
print("||w_final - w_true||:", float(np.linalg.norm(w_final - w_true)))

plt.figure()
for i in range(W_arr.shape[1]):
    plt.plot(W_arr[:, i], linewidth=1, label=f"w[{i}]")
plt.title("Trajetória dos coeficientes estimados")
plt.legend()
plt.show()

## Próximos passos (opcional)
- Transformar este notebook em um teste `pytest` (CI) com seed fixo.
- Adicionar verificação de estabilidade (todos os polos dentro do círculo unitário no final).